In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

img_size = 50
EPOCHS = 10 
BATCH_SIZE = 32

class Net(nn.Module):
   def __init__(self):
       super().__init__()
       self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
       self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
       self.conv3 = nn.Conv2d(64, 128, kernel_size=5)
       self.fc1 = nn.Linear(128*2*2, 512)
       self.fc2 = nn.Linear(512, 2)

   def forward(self, x):
       x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
       x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
       x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
       x = x.view(-1, 128*2*2)
       x = F.relu(self.fc1(x))
       x = self.fc2(x)
       x = F.softmax(x, dim=1)
       return x

def train_model():
   device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
   net = Net().to(device)
   optimizer = optim.Adam(net.parameters(), lr=0.001)
   loss_function = nn.MSELoss()
   
   X_train = np.load("melanoma_X_train.npy")
   y_train = np.load("melanoma_y_train.npy")
   
   train_X = torch.Tensor(X_train).to(device)
   train_X = train_X / 255.0
   train_y = torch.Tensor(y_train).to(device)
   
   for epoch in range(EPOCHS):
       running_loss = 0.0
       for i in range(0, len(train_X), BATCH_SIZE):
           batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, img_size, img_size)
           batch_y = train_y[i:i+BATCH_SIZE]
           
           optimizer.zero_grad()
           outputs = net(batch_X)
           loss = loss_function(outputs, batch_y)
           loss.backward()
           optimizer.step()
           
           running_loss += loss.item()
           
       epoch_loss = running_loss / (len(train_X) / BATCH_SIZE)
       print(f'Epoch {epoch+1}/{EPOCHS}. Loss: {epoch_loss:.4f}')
   
   torch.save(net.state_dict(), 'saved_model.pth')
   return net, device

def evaluate_model(net=None, device=None):
   if net is None or device is None:
       device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
       net = Net().to(device)
       net.load_state_dict(torch.load('saved_model.pth'))
   
   net.eval()
   
   X_test = np.load("melanoma_X_test.npy")
   y_test = np.load("melanoma_y_test.npy")
   
   test_X = torch.Tensor(X_test).to(device)
   test_X = test_X / 255.0
   test_y = torch.Tensor(y_test).to(device)
   
   correct = 0
   total = 0
   
   with torch.no_grad():
       outputs = net(test_X.view(-1, 1, img_size, img_size))
       predictions = (outputs[:, 0] >= outputs[:, 1]).float()
       labels = (test_y[:, 0] >= test_y[:, 1]).float()
       correct = (predictions == labels).sum().item()
       total = len(test_X)
           
   accuracy = round(correct/total, 3)
   print(f"Accuracy: {accuracy}")
   return accuracy

if __name__ == "__main__":
   net, device = train_model()
   evaluate_model(net, device)

Epoch 1/10. Loss: 0.1619
Epoch 2/10. Loss: 0.1232
Epoch 3/10. Loss: 0.1138
Epoch 4/10. Loss: 0.1067
Epoch 5/10. Loss: 0.1031
Epoch 6/10. Loss: 0.0977
Epoch 7/10. Loss: 0.0940
Epoch 8/10. Loss: 0.0909
Epoch 9/10. Loss: 0.0886
Epoch 10/10. Loss: 0.0829
Accuracy: 0.879
